In [1]:
import os
import urllib.request
import matplotlib.pyplot as plt
from scipy import spatial
from sklearn.manifold import TSNE
import numpy as np


In [2]:
urllib.request.urlretrieve('https://nlp.stanford.edu/data/glove.6B.zip','glove.6B.zip')

('glove.6B.zip', <http.client.HTTPMessage at 0x7f2a520a5250>)

In [3]:
!unzip "/content/glove.6B.zip" -d "/content/"

Archive:  /content/glove.6B.zip
  inflating: /content/glove.6B.50d.txt  
  inflating: /content/glove.6B.100d.txt  
  inflating: /content/glove.6B.200d.txt  
  inflating: /content/glove.6B.300d.txt  


In [4]:
embed_dict = {}
with open('/content/glove.6B.200d.txt','r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    embed_dict[word]=vector
  

In [5]:
def find_similar_word(emmbedes):
  nearest = sorted(embed_dict.keys(), key=lambda word: spatial.distance.euclidean(embed_dict[word], emmbedes))
  return nearest

In [7]:
import numpy as np
import pandas as pd
import nltk
import spacy
import os
import string
import gensim

In [8]:
extracted_questions = "python_queries.csv"
#df = pd.read_csv(join(path_prefix, extracted_questions))
df = pd.read_csv("python_queries.csv")
df.dropna()
df.head()

,"""tags""",is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,link,title,body,code
0,['python'; 'pandas'],True,40197,5,220,1649006979,1384990299,1.495540e+09,20109391,CC BY-SA 3.0,https://stackoverflow.com/questions/20109391/h...,How to make good reproducible pandas examples,Having spent a decent amount of time watching ...,[<code>pandas</code><code>pandas</code><code>i...
1,['python'; 'if-statement'; 'comparison'; 'matc...,True,425450,28,783,1642880434,1361967983,1.640170e+09,15112125,CC BY-SA 4.0,https://stackoverflow.com/questions/15112125/h...,How to test multiple variables for equality ag...,I'm trying to make a function that will compar...,[<code>x=0y=1z=3mylist=[]ifxoryorz==0:mylist.a...
2,['python'],True,850508,22,694,1644802105,1398432707,1.617213e+09,23294658,CC BY-SA 4.0,https://stackoverflow.com/questions/23294658/a...,Asking the user for input until they give a va...,I am writing a program that accepts an input f...,[<code>#note:Python2.7usersshoulduse`raw_input...
3,['python'; 'variable-variables'],True,207477,17,483,1648887168,1251981468,1.610503e+09,1373164,CC BY-SA 4.0,https://stackoverflow.com/questions/1373164/ho...,How do I create variable variables?,How do I accomplish variable variables in Pyth...,[]
4,['python'; 'slice'; 'sequence'],True,2520788,34,4192,1648552222,1233700262,1.648552e+09,509211,CC BY-SA 4.0,https://stackoverflow.com/questions/509211/und...,Understanding slicing,I need a good explanation (references are a pl...,[]


In [53]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
nltk.download('punkt')
nltk.download('stopwords')
def prepareString(input):
    stop_words = set(stopwords.words('english'))
    add_stopword = ['python', 'how', 'to', 'show', 'shows', 'showing', 'I']
    title = re.sub(r'[^\w\s]', '', input)
    title = title.lower()
    title = input.split(" ")
    
    # word_tokens = word_tokenize(title)
    filtered = [w for w in title if not (( w in stop_words) or ( w in add_stopword))]
    out= ""
    for w in filtered:
      out = out + w + " "
    return out
prepareString("I want to check if my objects are colliding")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'want check objects colliding '

In [9]:
import gensim
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec

PATH_TO_GLOVE = "glove.6B.300d.txt"

tmp_file = "/glove.840B.300d.w2v.txt"
glove2word2vec(PATH_TO_GLOVE, tmp_file)
glove = gensim.models.KeyedVectors.load_word2vec_format(tmp_file)

In [12]:
import scipy

def cosine_distance(s1, s2):
    vector_1 = np.mean([glove[word] for word in s1 if word in glove],axis=0)
    vector_2 = np.mean([glove[word] for word in s2 if word in glove],axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    return 1-cosine

In [19]:
df = pd.read_csv("python_queries.csv")
features_Title = ["question_id", "title"]
title = df["title"].values 
title = title[:10000]
print(type(title))

<class 'numpy.ndarray'>


In [20]:
q1 = "I want to check if my objects are colliding"
q2 = "How to check colliding objects"

cosine_distance(q1,q2)

0.9880519509315491

In [68]:
max = {}

for i in range(0,1000):
  
  cos =cosine_distance(prepareString(q1), prepareString(title[i]))
  max[title[i]] = cos

sort = sorted(max.items(), key=lambda x: x[1], reverse=True)

for i in range(0,5):
  print(sort[i])


('How to detect collisions between two rectangular objects or images in pygame', 0.9798070788383484)
('How do I watch a file for changes?', 0.9794266223907471)
('Spawning multiple instances of the same object concurrently in python', 0.9785071015357971)
('Shuffling a list of objects', 0.9782987236976624)
('How does collections.defaultdict work?', 0.977897047996521)
